## LIME と SHAP によるデータセット分析

In [1]:
import pandas as pd
import python.data_manager as dm
import importlib

2024-05-28 20:12:13,875 : python.data_manager : INFO : 32 : Test_message


In [2]:
importlib.reload(dm)

2024-05-28 20:12:13,898 : python.data_manager : INFO : 32 : Test_message


<module 'python.data_manager' from 'c:\\Users\\zigza\\OneDrive\\ドキュメント\\0_大学院\\Projects\\Test_LIME_SHAP\\src\\python\\data_manager.py'>

In [3]:
left_df  = dm.read_folder("../data/Reflection/")
right_df = dm.read_folder("../data/Grade/")

2024-05-28 20:12:13,919 : python.data_manager : INFO : 77 : read ..\data\Reflection *.csv data...
2024-05-28 20:12:13,921 : python.data_manager : INFO : 80 : read : [WindowsPath('../data/Reflection/Course_C-2021-1_Reflection.csv'), WindowsPath('../data/Reflection/Course_C-2021-2_Reflection.csv'), WindowsPath('../data/Reflection/Course_C-2022-1_Reflection.csv')]
2024-05-28 20:12:14,053 : python.data_manager : INFO : 87 : get 28275 rows.
2024-05-28 20:12:14,054 : python.data_manager : INFO : 77 : read ..\data\Grade *.csv data...
2024-05-28 20:12:14,055 : python.data_manager : INFO : 80 : read : [WindowsPath('../data/Grade/Course_C-2021-1_GradePoint.csv'), WindowsPath('../data/Grade/Course_C-2021-2_GradePoint.csv'), WindowsPath('../data/Grade/Course_C-2022-1_GradePoint.csv')]
2024-05-28 20:12:14,061 : python.data_manager : INFO : 87 : get 378 rows.


In [4]:
left_df.head()

,userid,course_number,question_number,question_content,answer_content
0,C-2021-1_U57,1,1,今日の内容を自分なりの言葉で説明してみてください,携帯やpcは音声、文章、写真を伝える。現在の通信技術に至るまでに、さまざまな技術が生み出され...
1,C-2021-1_U57,1,2,今日の内容で、分かったこと・できたことを書いてください,情報は１と０で表され、その組み合わせで伝えられる。
2,C-2021-1_U57,1,3,今日の内容で、分からなかったこと・できなかったことを書いてください,特になし。
3,C-2021-1_U57,1,4,質問があれば書いてください,特になし。
4,C-2021-1_U57,1,5,今日の授業の感想や反省を書いてください,特になし。


In [5]:
key = "userid"
df = pd.merge(left=left_df, right=right_df, on=key)

In [6]:
df.head()

,userid,course_number,question_number,question_content,answer_content,grade
0,C-2021-1_U57,1,1,今日の内容を自分なりの言葉で説明してみてください,携帯やpcは音声、文章、写真を伝える。現在の通信技術に至るまでに、さまざまな技術が生み出され...,B
1,C-2021-1_U57,1,2,今日の内容で、分かったこと・できたことを書いてください,情報は１と０で表され、その組み合わせで伝えられる。,B
2,C-2021-1_U57,1,3,今日の内容で、分からなかったこと・できなかったことを書いてください,特になし。,B
3,C-2021-1_U57,1,4,質問があれば書いてください,特になし。,B
4,C-2021-1_U57,1,5,今日の授業の感想や反省を書いてください,特になし。,B


In [7]:
_, dump = dm.data_clean(df)
dump.head()

2024-05-28 20:12:14,177 : python.data_manager : INFO : 218 : df shape : (28275, 6)
2024-05-28 20:12:14,179 : python.data_manager : INFO : 219 : cleaned_df shape : (20291, 6)
2024-05-28 20:12:14,181 : python.data_manager : INFO : 220 : dumped 7984 rows.
2024-05-28 20:12:14,206 : python.data_manager : INFO : 228 : dump_df shape : (7984, 6)


,userid,course_number,question_number,question_content,answer_content,grade
18,C-2021-1_U57,4,4,質問があれば書いてください,NaN,B
28,C-2021-1_U57,6,4,質問があれば書いてください,NaN,B
33,C-2021-1_U57,7,4,質問があれば書いてください,NaN,B
42,C-2021-1_U57,9,3,今日の内容で、分からなかったこと・できなかったことを書いてください,NaN,B
43,C-2021-1_U57,9,4,質問があれば書いてください,NaN,B


In [8]:
train, valid, test = dm.sprit_data_for_user(df, key=key, text="answer_content", label="grade", split_rate=[0.8, 0.1, 0.1])

In [9]:
print(f"""
      columns =
      train : {train.columns}
      valid : {valid.columns}
      test  : {test.columns}
      """
)


      columns =
      train : Index(['userid', 'grade', 'text', 'course_number', 'question_number'], dtype='object')
      valid : Index(['userid', 'grade', 'text', 'course_number', 'question_number'], dtype='object')
      test  : Index(['userid', 'grade', 'text', 'course_number', 'question_number'], dtype='object')
      


In [10]:
print(f"""
      rows =
      train : {len(train)}\t- {len(train) / (len(train) + len(valid) + len(test))} %
      valid : {len(valid)}\t- {len(valid) / (len(train) + len(valid) + len(test))} %
      test  : {len(test)}\t- {len(test ) / (len(train) + len(valid) + len(test))} %
      """
)


      rows =
      train : 22575	- 0.7984084880636605 %
      valid : 2850	- 0.10079575596816977 %
      test  : 2850	- 0.10079575596816977 %
      


### LIME実装